In [5]:
from embedding_model import EmbeddingModel
from typing import List
from transformers import AutoTokenizer, AutoModel, BertModel
import torch
from discourse import Discourse

In [1]:

from discourse import Discourse
from evaluate import load

In [12]:
sent = ["The sky is blue", "the weather is good", "the air is warm"]

In [3]:
metric = load("bertscore")


In [13]:
dis = Discourse(sent)
bertscore = dis.coherence_score(metric=metric)


In [14]:
bertscore

0.9103735983371735

In [34]:
bertscore2 = metric.compute(predictions=sent[1:], references=sent[:-1], lang="en")

In [38]:
type(bertscore2)

dict

In [27]:
sent = ["I live in Germany", "I study computer science"] 

In [60]:
encoded_input = bert_tokenizer(sent, padding=True, truncation=True, return_tensors='pt')
encoded_input

{'input_ids': tensor([[ 101, 1045, 2444, 1999, 2762,  102],
        [ 101, 1045, 2817, 3274, 2671,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1]])}

In [66]:
bert_model(**encoded_input).keys()

odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states', 'attentions'])

In [67]:
from collections import defaultdict


In [75]:
idf_dict_hyp = defaultdict(lambda x: 0*x + 1.)

In [78]:
refs = ['The dog bit the man.', 'The dog had bit the man.']
sys = 'The dog bit the man.'

In [81]:
idf_dict_hyp = defaultdict(lambda: 1.)
idf_dict_ref = defaultdict(lambda: 1.)

### Reading JSON File

In [4]:
import pandas as pd
import json

In [5]:
file = open("../data/train_sessions.json", "r", encoding="utf8")
data = json.loads(file.read())

In [49]:
# normalized_data = pd.json_normalize(data, record_path=["turns", "passage"], 
#                                     meta=["session_id", ["turns", "qid"], ["turns", "query"], ["turns", "oracle_query"], ["turns", "answer"]])

In [6]:
normalized_data = pd.json_normalize(data, record_path=["turns"], 
                                    meta=["session_id"])

In [7]:
session_ids = normalized_data["session_id"].unique()

In [8]:
queries = normalized_data[normalized_data['session_id'] == session_ids[300]]['oracle_query'].tolist()

In [11]:
queries

['What is Venezuela?',
 'What is the capital and the largest city of colombia?',
 'What is the capital of peru?',
 'What is the capital of bolivia?',
 'What is the capital of chile?']

### Testing with Metrics

In [7]:
from evaluate import load 

In [8]:
from evaluate import load
bertscore = load("bertscore")
bleu = load("bleu")
rouge = load('rouge')

In [12]:
queries.append("blue sky in summer")

In [18]:
queries.pop(-1)

'blue sky in summer'

In [19]:
queries

['What is Venezuela?',
 'What is the capital and the largest city of colombia?',
 'What is the capital of peru?',
 'What is the capital of bolivia?',
 'What is the capital of chile?']

In [13]:
bertscore.compute(predictions=queries[1:], references=queries[:-1], lang="en", model_type="microsoft/deberta-xlarge-mnli")

{'precision': [0.5483058094978333,
  0.8842901587486267,
  0.9398562908172607,
  0.9501819610595703,
  0.5051981806755066],
 'recall': [0.6928839087486267,
  0.7621741890907288,
  0.9398562908172607,
  0.9501819610595703,
  0.5498039126396179],
 'f1': [0.612174391746521,
  0.8187035918235779,
  0.9398562908172607,
  0.9501819610595703,
  0.5265581011772156],
 'hashcode': 'microsoft/deberta-xlarge-mnli_L40_no-idf_version=0.3.12(hug_trans=4.36.2)'}

In [20]:
rouge.compute(predictions=queries[1:], references=queries[:-1])

{'rouge1': 0.6498397435897436,
 'rouge2': 0.5525974025974026,
 'rougeL': 0.6498397435897436,
 'rougeLsum': 0.6498397435897436}

In [21]:
bleu.compute(predictions=queries[1:], references=queries[:-1])

{'bleu': 0.3912711450183218,
 'precisions': [0.65625, 0.42857142857142855, 0.3333333333333333, 0.25],
 'brevity_penalty': 1.0,
 'length_ratio': 1.103448275862069,
 'translation_length': 32,
 'reference_length': 29}

## Testing with DiscoScore

In [22]:
from disco_score import DiscoScorer

Downloaded pre-trained UDPipe model for 'en' language


In [24]:
disco_scorer = DiscoScorer(model_name='bert-base-uncased')

AssertionError: Torch not compiled with CUDA enabled

## Testing with AnglE

In [10]:
from angle_emb import AnglE, Prompts

In [11]:
angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()
angle.set_prompt(prompt=Prompts.C)

INFO:AnglE:Prompt is set, the prompt will be automatically applied during the encoding phase. To disable prompt setting, please configure set_prompt(prompt=None)


In [39]:
queries_dic = [{'text': query} for query in queries]
embeddings = angle.encode(queries_dic)


In [36]:
queries.append("the sky is blue")
queries

['What is Venezuela?',
 'What is the capital and the largest city of colombia?',
 'What is the capital of peru?',
 'What is the capital of bolivia?',
 'What is the capital of chile?',
 'the sky is blue',
 'the sky is blue']

## Trying with nouns only

In [23]:
from utils import *

In [33]:
sent1 = "I am a blue cloud person in germany"
sent2 = "the sky is blue"

In [30]:
similarity("cloud", "sky")

0.74419004

In [34]:
similarity(sent1, sent2)

0.7236823

In [25]:
similarity(get_nouns(sent1), get_nouns(sent2))

0.4209405

In [12]:
emb = angle.encode({"text": "Hello over there"})

In [16]:
from moverscore_v2 import get_idf_dict, word_mover_score 
from collections import defaultdict

SyntaxError: invalid syntax (moverscore_v2.py, line 7)